In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation

from tensorflow.keras.models import Sequential
from keras import optimizers
from tensorflow.keras.losses import MSE

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.metrics import mean_squared_error

df=pd.read_csv('/dataset.csv')
test=pd.read_csv('/test.csv')

In [19]:
df.rename(columns={'표층 수온 (℃)':'SST'},inplace=True)
train=df.copy()
train.iloc[48]

test=test.sort_values(['관측일시','lat','long']).reset_index(drop=True)
sst=train['SST']

train.pop('관측소명')
train.pop('관측일시')
reshape_dataset=train.values.reshape(train.shape)
reshape_dataset

test.pop('관측일시')
test.pop('관측소명')
reshape_test=test.values.reshape(test.shape)
reshape_test

array([[ 17.3   ,  33.1628, 126.2639],
       [ 16.4   ,  33.2253, 126.3931],
       [ 17.2   ,  33.2386, 126.1947],
       ...,
       [ 17.    ,  33.5222, 126.9667],
       [ 17.    ,  33.5253, 126.4939],
       [ 16.4   ,  33.5817, 126.7636]])

In [20]:
seq_len=15
steps=1
X,y=list(),list()
for i, _ in  enumerate(reshape_dataset):
  idx_in=i+seq_len
  idx_out= idx_in + steps
  if idx_out>len(reshape_dataset):
    break
  seq_x=reshape_dataset[i:idx_in,1:]
  # print(seq_x)
  seq_y=reshape_dataset[idx_out-1:idx_out,0]
  # print(seq_y)

  X.append(seq_x)
  y.append(seq_y)
X=np.array(X)
y=np.array(y)

In [21]:
epochs = 100
batch = 64
lr = 0.01
adam = optimizers.Adam(lr)

In [22]:
tf.random.set_seed(42)
model=Sequential()
model.add(LSTM(100, activation='relu', input_shape=(seq_len, 2), return_sequences=True))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer=adam)


In [ ]:
lstm_history = model.fit(X, y,  epochs=epochs, verbose=2)

Epoch 1/100
4380/4380 - 48s - loss: 164.8964 - 48s/epoch - 11ms/step
Epoch 2/100
4380/4380 - 46s - loss: 23.6807 - 46s/epoch - 11ms/step
Epoch 3/100
4380/4380 - 46s - loss: 20.2042 - 46s/epoch - 10ms/step
Epoch 4/100
4380/4380 - 46s - loss: 20.0855 - 46s/epoch - 10ms/step
Epoch 5/100
4380/4380 - 46s - loss: 21.2070 - 46s/epoch - 11ms/step
Epoch 6/100
4380/4380 - 46s - loss: 20.1398 - 46s/epoch - 10ms/step
Epoch 7/100
4380/4380 - 46s - loss: 20.1088 - 46s/epoch - 11ms/step
Epoch 8/100
4380/4380 - 46s - loss: 20.0983 - 46s/epoch - 10ms/step
Epoch 9/100
4380/4380 - 46s - loss: 20.0976 - 46s/epoch - 11ms/step
Epoch 10/100
4380/4380 - 46s - loss: 20.0018 - 46s/epoch - 10ms/step
Epoch 11/100
4380/4380 - 46s - loss: 19.9811 - 46s/epoch - 11ms/step
Epoch 12/100


In [16]:
pred_train=model.predict(X)
y=y.flatten()
np.sqrt(mean_squared_error(pred_train,y))

4379/4379 [==============================] - 48s 11ms/step


4.632145519271082

In [17]:
pred_test=model.predict(X_test)
y_test=y_test.flatten()
print(np.sqrt(mean_squared_error(pred_test,y_test)))

4379/4379 [==============================] - 47s 11ms/step
4.632145519271082
